# Controlling symbolic music generation with event attribute constraints & superposed language modelling

This notebook demonstrates three examples of constrained music generation using the SLM (Superposed Language Modelling) framework.

In [1]:
# Colab setup - install requirements if running in Google Colab
import os
import sys

# Detect if running in Colab by checking if the repo exists locally
IS_COLAB = not os.path.exists("../slm")

if IS_COLAB:
    print("Running in Google Colab - setting up environment...")
    
    # Clone the repository
    !git clone https://github.com/erl-j/superposed-language-modelling
    
    # Install the package in editable mode
    !pip install -e superposed-language-modelling
    
    # Add the package to Python path immediately
    sys.path.insert(0, '/content/superposed-language-modelling')
    
    print("Setup complete!")
else:
    print("Running locally - skipping setup")

Running locally - skipping setup


## Setup and Imports

Import necessary libraries and load dependencies.

In [ ]:
import random
from pathlib import Path
import torch
import torch
import fractions
import symusic
torch.serialization.add_safe_globals([fractions.Fraction])
from slm.util import preview_sm, sm_fix_overlap_notes, loop_sm, plot_piano_roll, preview_w_player
from slm.conversion_utils import sm_to_events
from slm.constraints.core import (
    MusicalEventConstraint,
    DRUM_PITCHES,
    PERCUSSION_PITCHES,
    TOM_PITCHES,
    CRASH_PITCHES,
    HIHAT_PITCHES,
    RIDE_PITCHES,
)
import slm.util as util
from slm import load_model

SF_PATH = util.get_matrix_soundfont_path()

# Configuration
USE_FP16 = False
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Model Loading and Helper Functions

Load the model and define helper functions to simplify the generation process.

In [ ]:
print(f"Loading model on {DEVICE}...")

# Download checkpoint
model = load_model(
    model_type="slm_mixed",
    epochs=150,
    device=DEVICE
)

if USE_FP16:
    model = model.convert_to_half()

N_EVENTS = model.tokenizer.config["max_notes"]

# Helper to convert constraints to music
def generate_from_constraints(e, sampling_params={}):
    print("Generating...")
    mask = model.tokenizer.event_constraints_to_mask(e).to(DEVICE)
    x = model.generate(
        mask,
        temperature=sampling_params.get("temperature", 1.0),
        top_p=sampling_params.get("top_p", 1.0),
        top_k=sampling_params.get("top_k", 0),
        tokens_per_step=sampling_params.get("tokens_per_step", 1),
        attribute_temperature=sampling_params.get("attribute_temperature", None),
        order=sampling_params.get("order", "random"),
        collapse_duplicates=True,
        constraint_cfg=sampling_params.get("constraint_cfg", 1.0),
    )[0].argmax(-1)
    x_sm = model.tokenizer.decode(x)
    notes_before = x_sm.note_num()
    x_sm = util.sm_fix_overlap_notes(x_sm)
    notes_after = x_sm.note_num()
    print(f"Removed {notes_before - notes_after} overlapping notes.")
    return x_sm

# Helper to convert existing symusic score (sm) to a constraint list
def sm_to_constraint(sm, tag="other"):
    return sm_to_events(sm, tag, tokenizer=model.tokenizer)

# Short alias for creating a new constraint object
ec = lambda: MusicalEventConstraint(model.tokenizer)

print("Ready to generate!")


Loading model on cuda...
Loading model from /root/.cache/slm/slm_mixed_150epochs.ckpt...
Ready to generate!


## Example 1: Generation from detailed constraints

This constraint specifies:
- **12 kick drums** (pitch 36) distributed throughout the sequence
- **4 snare drums** (pitch 38)
- **24 ride cymbal or hi-hat** 
- **2-6 tom drums** constrained to the last two bars (ticks 192-384) for a tom fill.
- **16 required + 16 optional guitar notes**
- **16 bass notes** in G pentatonic scale (pitch range 27-45) with durations of 1/8 or 1/16 notes
- **4 optional bass notes**
- **Tempo fixed at 130 BPM**

In [ ]:
SEED = 0

# Set seeds for reproducibility
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    
def constraint_rock_loop(e, ec):
    e = []
    # add 12 kicks
    e += [ec().intersect({"pitch": {"36 (Drums)"}}).force_active() for _ in range(12)]
    # add 4 snares
    e += [ec().intersect({"pitch": {"38 (Drums)"}}).force_active() for _ in range(4)]
    # add 24 ride cymbals or hats 
    e += [ec().intersect({"pitch": RIDE_PITCHES}).force_active() for _ in range(24)]
    # add between 2 and 6 toms in last two bars
    last_bar_ticks = {str(t) for t in range(192, 384)}
    e += [ec().intersect({"pitch": TOM_PITCHES, "onset/global_tick": last_bar_ticks}).force_active() for _ in range(2)]
    e += [ec().intersect({"pitch": TOM_PITCHES, "onset/global_tick": last_bar_ticks}) for _ in range(6)]
    # add 46 guitar notes
    e += [
        ec()
        .intersect({"instrument": {"Guitar"}})
        .force_active()
        for _ in range(16)
    ]
    e += [
        ec()
        .intersect({"instrument": {"Guitar"}})
        for _ in range(16)
    ]
    # add 10 bass notes with duration and pitch constraint
    e += [
        ec()
        .intersect({"instrument": {"Bass"}, "duration":{ "1/8", "1/16"}})
        .intersect(ec().pitch_in_scale_constraint("G pentatonic", (27, 45)))
        .force_active()
        for _ in range(16)
    ]
    # add 4 optional bass notes
    e += [
        ec()
        .intersect({"instrument": {"Bass"}})
        for _ in range(4)
    ]
    # pad with empty notes
    e += [ec().force_inactive() for _ in range(N_EVENTS - len(e))]
    # set tempo to 160
    e = [ev.intersect(ec().tempo_constraint(130)) for ev in e]
    # set tag to rock
    return e

# Generate from constraints
constraints = constraint_rock_loop([], ec)

sm = generate_from_constraints(constraints, {"temperature": 1.0, "top_p": 1.0, "constraint_cfg": 1.0, "tokens_per_step": 1, "top_k": 0, "order": "random", "attribute_temperature": None})
preview_w_player(sm)
sm.dump_midi("rock_loop.mid")


Generating...


100%|██████████| 658/658 [00:10<00:00, 64.83it/s]

Removed 3 overlapping notes.


## Example 2: Box infill (inpainting)

**What this constraint expresses:**

This constraint performs selective inpainting on existing music:
- Takes the previously generated rock loop as input
- Defines a **"box"** in time-pitch space: tick range (96, 288) and pitch range (34, 58)
- **Removes or modifies** notes that fall within or overlap this box:
  - Notes entirely inside the box are deleted
  - Notes that span across box boundaries have their onsets/offsets resampled
  - Notes outside the box are preserved unchanged
- **Adds 18 new events** within the box region with appropriate instrument, pitch, and timing constraints
- All drum notes are excluded from modification (drums=False)

This allows surgical editing of specific time-pitch regions.

In [ ]:
def constraint_box_infill(
    e,
    ec,
    tick_range,
    pitch_range,
    drums,
    n_new_events,
):
    e = [ev for ev in e if not ev.is_inactive()]
    instruments = ec().a["instrument"]
    if not instruments:
        instruments = ec().a["instrument"]

    box_ticks = set(range(tick_range[0], tick_range[1]))
    valid_onsets = {str(r) for r in box_ticks}
    valid_offsets = {str(r) for r in range(tick_range[0] + 1, tick_range[1] + 1)}
    valid_pitches = {
        f"{r} (Drums)" if drums else f"{r}"
        for r in range(pitch_range[0], pitch_range[1])
    }
    def _min_tick(values):
        ints = [int(v) for v in values if v.isdigit()]
        return min(ints) if ints else None
    def _max_tick(values):
        ints = [int(v) for v in values if v.isdigit()]
        return max(ints) if ints else None
    def _pitch_in_box(pitch_set):
        return bool(pitch_set & valid_pitches)
    filtered = []
    for ev in e:
        start = _min_tick(ev.a["onset/global_tick"])
        end = _max_tick(ev.a["offset/global_tick"])
        pitch_in = _pitch_in_box(ev.a["pitch"])
        # only consider notes that overlap the pitch range
        if not pitch_in:
            filtered.append(ev)
            continue
        # check time overlap with box
        starts_before = start is not None and start < tick_range[0]
        starts_in = start is not None and tick_range[0] <= start < tick_range[1]
        ends_in = end is not None and tick_range[0] < end <= tick_range[1]
        ends_after = end is not None and end > tick_range[1]
        # note entirely in box -> delete
        if starts_in and ends_in:
            continue
        # note spans the entire box (starts before, ends after) -> delete
        if starts_before and ends_after:
            continue
        # note starts before and ends in box -> resample offset
        if starts_before and ends_in:
            ev.a["offset/global_tick"] = valid_offsets.copy()
            filtered.append(ev)
            continue
        # note starts in box and ends outside -> resample onset
        if starts_in and ends_after:
            ev.a["onset/global_tick"] = valid_onsets.copy()
            filtered.append(ev)
            continue
        # note doesn't touch the box at all
        filtered.append(ev)
    infill_constraint = {
        "pitch": valid_pitches | {"-"},
        "onset/global_tick": valid_onsets | {"-"},
        "offset/global_tick": valid_offsets | {"-"},
        "instrument": ({"Drums"} if drums else instruments - {"Drums"}) | {"-"},
    }
    # force at least one event
    filtered += [ec().intersect(infill_constraint).force_active() for _ in range(n_new_events)]
    # pad with inactive notes
    filtered += [ec().force_inactive() for _ in range(N_EVENTS - len(filtered))]
    return filtered

# Load the previously generated rock loop
sm_source = symusic.Score("rock_loop.mid")
source_constraints = sm_to_constraint(sm_source)

# Generate infilled version
tick_range = (96, 288)
pitch_range = (34, 58)
infill_constraints = constraint_box_infill(
    source_constraints, 
    ec, 
    tick_range=tick_range, 
    pitch_range=pitch_range, 
    drums=False, 
    n_new_events=18
)
sm = generate_from_constraints(infill_constraints, {"temperature": 1.0, "top_p": 1.0})
preview_w_player(sm)

Generating Box Inpainting Example...
Generating...


100%|██████████| 148/148 [00:02<00:00, 64.33it/s]

Removed 0 overlapping notes.


## Example 3: Generation from unusual constraints

This constraint specifies:
- **32 chromatic percussion notes** at 1/4 note duration, on half-note triplets (every 16 ticks from tick 16 to 384), using the A# kaweco scale (pitch range 24-108)
- **10 percussion hits** from the drum kit's percussion subset
- **10 soft percussion notes** (velocity < 50) 
- **10 loud percussion notes** (velocity > 100)
- **16 ensemble/flute notes** with 1/16 duration
- **8 bass notes** with 1/8 duration
- **Tempo set to 100 BPM**

In [ ]:
def constraint_unusual(e, ec):
    e = []
    # add some chromatic percussion
    e += [ec().intersect({"instrument": {"Chromatic Percussion"}, "duration":{"1/4"}, "onset/global_tick": {str(tick) for tick in range(16, 384, 16)}})
    .intersect(ec().pitch_in_scale_constraint("A# kaweco",(24, 108))).force_active() for i in range(32)]
    # add ten percussion
    e += [ec().intersect({"instrument": {"Drums"}, "pitch": PERCUSSION_PITCHES}).force_active() for i in range(10)]
    # add 30 percussion pitches with velocity less than 50
    e += [ec().intersect({"instrument": {"Drums"}, "pitch": PERCUSSION_PITCHES}).intersect(ec().velocity_constraint(50)).force_active() for i in range(10)]
    # add 30 percussion pitches with velocity greater than 100
    e += [ec().intersect({"instrument": {"Drums"}, "pitch": PERCUSSION_PITCHES}).intersect(ec().velocity_constraint(100)).force_active() for i in range(10)]
    # add some pipe flute
    e += [ec().intersect({"instrument": {"Ensemble"}, "duration": {"1/16"}}).force_active() for i in range(16)]
    # add some synth bass
    e += [ec().intersect({"instrument": {"Bass"}, "duration": {"1/8"}}).force_active() for i in range(8)]
    # set tempo to 130
    e = [ev.intersect(ec().tempo_constraint(100)) for ev in e]
    # pad remaining space
    e += [ec().force_inactive() for _ in range(N_EVENTS - len(e))]
    return e

# Generate from constraints
constraints = constraint_unusual([], ec)
sm = generate_from_constraints(constraints)
preview_w_player(sm)

Generating...


100%|██████████| 440/440 [00:06<00:00, 64.41it/s]

Removed 20 overlapping notes.
